<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# LSTM para series de tiempo
## *Modelos no lineales para pronósitico*  - Pedro Martinez

---

# Objetivo

Diseñar una red LSTM para predecir el comportamiento del índice S&P 500 utilizando datos reales de Yahoo Finance.

Además de construir el modelo, analizaremos el **efecto de los principales hiperparámetros** de la capa LSTM en Keras. Considerando:


1. Descargar y preparar datos financieros reales.  
2. Construir una red LSTM paso a paso.  
3. Comprender el papel de los hiperparámetros:  
    `units`, `return_sequences`, `dropout`, `recurrent_dropout`, `batch_size`, `epochs`, `optimizer`.  
4. Visualizar cómo cambian los resultados al modificarlos.

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [8]:

raw = pd.read_csv('data_raw/msft_raw.csv', header=None)
date_col_name = raw.iloc[2, 0]          

price_cols = raw.iloc[0, 1:].tolist()   
data = raw.iloc[3:, :].copy()
data.columns = [date_col_name] + price_cols

data[date_col_name] = pd.to_datetime(data[date_col_name])
data.set_index(date_col_name, inplace=True)

data = data.apply(pd.to_numeric, errors='coerce')


if isinstance(data.columns, pd.MultiIndex):
    data.columns = [' '.join(col).strip() for col in data.columns.values]

if 'Close' in data.columns:
    close_series = data['Close']
else:
    col_close = [c for c in data.columns if 'Close' in c][0]
    close_series = data[col_close]

close_series = pd.Series(close_series.values, index=pd.to_datetime(data.index))
close_series = pd.to_numeric(close_series, errors='coerce').dropna()


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=close_series.index,
    y=close_series.values,
    mode='lines',
    line=dict(color='royalblue'),
    name='Cierre MSFT'
))

fig.update_layout(
    title='MSFT — Precio de Cierre Diario',
    xaxis_title='Fecha',
    yaxis_title='Precio (USD)',
    hovermode='x unified',
    yaxis=dict(tickprefix='$', separatethousands=True)
)

fig.show()


1. Escalamiento de los datos.
Cual escalador podemos utilizar?

2. Definir entrada de datos. Como se pueden ingresar los datos a un LSTM?

3. Como se declara un modelos LSTM en Keras?